<a href="https://colab.research.google.com/github/maverick2903/SEM6_Programs/blob/main/ML/BayesianNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy -q

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

model = BayesianNetwork([('Burglary','Alarm'),('Earthquake','Alarm'),('Alarm','JohnCalls'),('Alarm','MaryCalls')])

# P(Burglary)
cpd_burglary = TabularCPD(variable='Burglary',variable_card=2, values=[[0.001], [0.999]])

# P(Earthquake)
cpd_earthquake = TabularCPD(variable='Earthquake',variable_card=2, values=[[0.002], [0.998]])

# P(Alarm | Burglary, Earthquake)
cpd_alarm = TabularCPD(variable='Alarm',variable_card=2, values=[[0.95,0.94,0.29,0.001],[0.05,0.06,0.71,0.999]],evidence=['Burglary', 'Earthquake'],evidence_card=[2,2])

# P(JohnCalls | Alarm)
cpd_johncalls = TabularCPD(variable='JohnCalls',variable_card=2, values=[
    [0.9, 0.05,], [0.10, 0.95,]
],evidence=['Alarm'],evidence_card=[2])

# P(MaryCalls | Alarm)
cpd_marycalls = TabularCPD(variable='MaryCalls',variable_card=2, values=[
    [0.7, 0.01,], [0.30, 0.99,]
],evidence=['Alarm'],evidence_card=[2])


model.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_johncalls, cpd_marycalls)

print(model)


BayesianNetwork with 5 nodes and 4 edges


In [ ]:
print("Burglary probability distribution")
print(cpd_burglary)
print()
print("Earthquake probability distribution")
print(cpd_earthquake)
print()
print("Burglary probability distribution")
print(cpd_alarm)
print()
print("Mary Calls probability distributioSn")
print(cpd_marycalls)
print()
print("John Calls probability distribution")
print(cpd_johncalls)

Burglary probability distribution
+-------------+-------+
| Burglary(0) | 0.001 |
+-------------+-------+
| Burglary(1) | 0.999 |
+-------------+-------+

Earthquake probability distribution
+---------------+-------+
| Earthquake(0) | 0.002 |
+---------------+-------+
| Earthquake(1) | 0.998 |
+---------------+-------+

Burglary probability distribution
+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.95          | 0.94          | 0.29          | 0.001         |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.05          | 0.06          | 0.71          | 0.999         |
+-----------

In [ ]:
print(f"Verify Model: {model.check_model()}")

Verify Model: True


In [ ]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
# Query 1: Find probability John and Mary calls and no burglary and no earthquake but alarm rings
query_1 = infer.query(variables=['JohnCalls','MaryCalls'], evidence={'Burglary':0,'Earthquake':0,'Alarm':1,})
print("Query 1: Find probability John and Mary calls and no burglary and no earthquake but alarm rings ")
print(query_1)
print(query_1.values[-1][-1])

# Query 2: Probability of John calls
query_2 = infer.query(variables=['JohnCalls'], evidence={})
print("\nQuery 2: Probability of John calls")
print(query_2)
print(query_2.values[-1])

# Query 3: Probability of burglary when both John and Mary calls
query_3 = infer.query(variables=['Burglary'], evidence={'JohnCalls': 1, 'MaryCalls': 1})
print("\nQuery 3: Probability of burglary when both John and Mary calls")
print(query_3)
print(query_3.values[-1])

Query 1: Find probability John and Mary calls and no burglary and no earthquake but alarm rings 
+--------------+--------------+----------------------------+
| JohnCalls    | MaryCalls    |   phi(JohnCalls,MaryCalls) |
+==============+==============+============================+
| JohnCalls(0) | MaryCalls(0) |                     0.0005 |
+--------------+--------------+----------------------------+
| JohnCalls(0) | MaryCalls(1) |                     0.0495 |
+--------------+--------------+----------------------------+
| JohnCalls(1) | MaryCalls(0) |                     0.0095 |
+--------------+--------------+----------------------------+
| JohnCalls(1) | MaryCalls(1) |                     0.9405 |
+--------------+--------------+----------------------------+
0.9405

Query 2: Probability of John calls
+--------------+------------------+
| JohnCalls    |   phi(JohnCalls) |
+==============+==================+
| JohnCalls(0) |           0.0521 |
+--------------+------------------+
| JohnCal